In [1]:
import os
import logging
import pandas as pd
import mysql.connector

In [14]:
MYSQL_USERNAME=os.environ['MYSQL_USERNAME']
MYSQL_PASSWORD=os.environ['MYSQL_PASSWORD']
MYSQL_DATABASE=os.environ['MYSQL_DATABASE']
MYSQL_HOST=os.environ['MYSQL_HOST']
# Connect to MySQL
try:
    # Establish connection
    conn = mysql.connector.connect(
        host=MYSQL_HOST,  # Change to your MySQL server
        user=MYSQL_USERNAME,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE  # Optional: specify database
    )

    if conn.is_connected():
        print("Connected to MySQL successfully!")

except mysql.connector.Error as e:
    print("Error:", e)

Connected to MySQL successfully!


In [15]:
# Create a cursor object
cursor = conn.cursor()

In [16]:
def extract(cursor, query):
    # Execute a query
    cursor.execute(query)
    data = cursor.fetchall()
    
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [17]:
query="SHOW TABLES;"
df = extract(cursor, query)

In [18]:
df

,Tables_in_OLTP_Database
0,Customer
1,Employee
2,Product
3,ProductCategory
4,ProductSubcategory
5,Promotion
6,Store
7,Transaction
8,TransactionDetail


In [19]:
query="SELECT * FROM Transaction;"
df = extract(cursor, query)
df

,TransactionID,CustomerID,StoreID,EmployeeID,TransactionDate,PromotionID,TotalPrice
0,1,1,1,2,2023-07-10 14:30:00,1.0,879.98
1,2,2,2,3,2023-07-11 10:15:00,NaN,89.99
2,3,3,3,4,2023-07-12 11:00:00,2.0,35.98
3,4,4,1,1,2023-07-13 16:45:00,NaN,799.99
4,5,5,1,2,2023-07-14 09:30:00,NaN,1109.98
5,6,6,2,3,2023-07-15 12:00:00,1.0,59.99
6,7,7,3,4,2023-07-16 14:00:00,NaN,20.00
7,8,8,1,2,2023-07-17 10:00:00,NaN,29.99
8,9,9,2,3,2023-07-18 11:15:00,2.0,799.99
9,10,10,3,4,2023-07-19 15:00:00,NaN,899.99


In [20]:
query="SELECT * FROM TransactionDetail;"
data = extract(cursor, query)
data

,TransactionDetailID,TransactionID,ProductID,Quantity,Price
0,1,1,1,1,799.99
1,2,1,2,1,999.99
2,3,1,3,1,59.99
3,4,2,4,1,89.99
4,5,3,7,2,31.98
5,6,4,1,1,799.99
6,7,5,1,1,899.99
7,8,5,3,1,59.99
8,9,6,4,1,89.99
9,10,7,5,1,49.99


In [21]:
# transactionid, customersk, employeesk, storesk, promotionsk, datesk, quantity, totalprice
query="""SELECT 
    t.TransactionID, 
    t.CustomerID, 
    t.EmployeeID, 
    td.ProductID, 
    t.StoreID, 
    t.PromotionID, 
    t.TransactionDate, 
    td.Quantity,
    t.TotalPrice 

FROM Transaction t
JOIN TransactionDetail td
ON t.TransactionID = td.TransactionID;
"""
#    td.Price
# td.TransactionDetailID, 

data = extract(cursor, query)
data

,TransactionID,CustomerID,EmployeeID,ProductID,StoreID,PromotionID,TransactionDate,Quantity,TotalPrice
0,1,1,2,1,1,1.0,2023-07-10 14:30:00,1,879.98
1,1,1,2,2,1,1.0,2023-07-10 14:30:00,1,879.98
2,1,1,2,3,1,1.0,2023-07-10 14:30:00,1,879.98
3,2,2,3,4,2,NaN,2023-07-11 10:15:00,1,89.99
4,3,3,4,7,3,2.0,2023-07-12 11:00:00,2,35.98
5,4,4,1,1,1,NaN,2023-07-13 16:45:00,1,799.99
6,5,5,2,1,1,NaN,2023-07-14 09:30:00,1,1109.98
7,5,5,2,3,1,NaN,2023-07-14 09:30:00,1,1109.98
8,6,6,3,4,2,1.0,2023-07-15 12:00:00,1,59.99
9,7,7,4,5,3,NaN,2023-07-16 14:00:00,1,20.00


In [29]:
data['TransactionDate'] = data['TransactionDate'].dt.strftime('%Y%m%d')
data

,TransactionID,CustomerID,EmployeeID,ProductID,StoreID,PromotionID,TransactionDate,Quantity,TotalPrice
0,1,1,2,1,1,1.0,20230710,1,879.98
1,1,1,2,2,1,1.0,20230710,1,879.98
2,1,1,2,3,1,1.0,20230710,1,879.98
3,2,2,3,4,2,NaN,20230711,1,89.99
4,3,3,4,7,3,2.0,20230712,2,35.98
5,4,4,1,1,1,NaN,20230713,1,799.99
6,5,5,2,1,1,NaN,20230714,1,1109.98
7,5,5,2,3,1,NaN,20230714,1,1109.98
8,6,6,3,4,2,1.0,20230715,1,59.99
9,7,7,4,5,3,NaN,20230716,1,20.00


In [31]:
# Close the connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.


In [32]:
import psycopg2
from psycopg2 import sql

In [33]:
PG_USERNAME=os.environ['PG_USERNAME']
PG_PASSWORD=os.environ['PG_PASSWORD']
PG_DATABASE=os.environ['PG_DATABASE']
PG_HOST=os.environ['PG_HOST']

In [34]:
# Database connection parameters
db_config = {
    'dbname': PG_DATABASE,
    'user': PG_USERNAME,
    'password': PG_PASSWORD,
    'host': PG_HOST,  # or your database host
    'port': 5432          # default PostgreSQL port
}

In [35]:
# Connect to the database
try:
    conn = psycopg2.connect(**db_config)
    print("Connected to the database!")
except Exception as e:
    print(f"Unable to connect to the database: {e}")


Connected to the database!


In [36]:
# Create a cursor object to interact with the database
cursor = conn.cursor()

In [37]:
def get_surrogate_key(df_bk, col_id, dim_table, business_column, surrogate_column):
    query = f"SELECT {business_column}, {surrogate_column} FROM {dim_table}"
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data, columns=[business_column, surrogate_column])
    key_map = dict(zip(df[business_column], df[surrogate_column]))
    return df_bk[col_id].map(key_map)

In [38]:
data['CustomerID'] = get_surrogate_key(data, 'CustomerID', 'dimcustomer', 'customerbk', 'customersk')
data['EmployeeID'] = get_surrogate_key(data, 'EmployeeID', 'dimemployee', 'Employeebk', 'employeesk')  
data['ProductID'] = get_surrogate_key(data, 'ProductID', 'dimproduct', 'Productbk', 'productSK')
data['StoreID'] = get_surrogate_key(data, 'StoreID', 'dimstore', 'storebk', 'storesk')
data['PromotionID'] = get_surrogate_key(data, 'PromotionID', 'dimpromotion', 'Promotionbk', 'promotionsk')


In [39]:
import numpy as np
data = data.replace({np.nan: None})
data

,TransactionID,CustomerID,EmployeeID,ProductID,StoreID,PromotionID,TransactionDate,Quantity,TotalPrice
0,1,232,1279,265,248,241.0,20230710,1,879.98
1,1,232,1279,267,248,241.0,20230710,1,879.98
2,1,232,1279,272,248,241.0,20230710,1,879.98
3,2,233,1280,274,249,None,20230711,1,89.99
4,3,234,1281,270,250,242.0,20230712,2,35.98
5,4,235,1278,265,248,None,20230713,1,799.99
6,5,236,1279,265,248,None,20230714,1,1109.98
7,5,236,1279,272,248,None,20230714,1,1109.98
8,6,237,1280,274,249,241.0,20230715,1,59.99
9,7,238,1281,275,250,None,20230716,1,20.00


In [ ]:
query = sql.SQL(f"""
    INSERT INTO factsales (transactionid, customersk, employeesk, productsk, storesk, promotionsk, datesk, quantity, totalprice)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
""")

In [41]:
def load(cursor, query, data):
    for i, row in data.iterrows():

        try:
            print(tuple(row))
            cursor.execute(query, tuple(row))
        
        except Exception as e:
            conn.rollback()  # Rollback in case of error
            print(f"Error loading factsales: {e}")

    cursor.execute('commit')
    print("Data Loaded successfully!")

In [42]:
load(cursor, query, data)

(1, 232, 1279, 265, 248, 241.0, '20230710', 1, Decimal('879.98'))
(1, 232, 1279, 267, 248, 241.0, '20230710', 1, Decimal('879.98'))
(1, 232, 1279, 272, 248, 241.0, '20230710', 1, Decimal('879.98'))
(2, 233, 1280, 274, 249, None, '20230711', 1, Decimal('89.99'))
(3, 234, 1281, 270, 250, 242.0, '20230712', 2, Decimal('35.98'))
(4, 235, 1278, 265, 248, None, '20230713', 1, Decimal('799.99'))
(5, 236, 1279, 265, 248, None, '20230714', 1, Decimal('1109.98'))
(5, 236, 1279, 272, 248, None, '20230714', 1, Decimal('1109.98'))
(6, 237, 1280, 274, 249, 241.0, '20230715', 1, Decimal('59.99'))
(7, 238, 1281, 275, 250, None, '20230716', 1, Decimal('20.00'))
(8, 239, 1279, 278, 248, None, '20230717', 1, Decimal('29.99'))
(9, 240, 1280, 265, 249, 242.0, '20230718', 1, Decimal('799.99'))
(10, 241, 1281, 270, 250, None, '20230719', 1, Decimal('899.99'))
(11, 242, 1279, 280, 248, None, '20230801', 1, Decimal('1399.99'))
(12, 243, 1280, 282, 249, 241.0, '20230802', 1, Decimal('99.99'))
(13, 244, 1281, 28

In [43]:
# Close the cursor and connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.
